# Notebook 2 - Searching for solutions

CSI4106 Artificial Intelligence  
Fall 2021  
Version 1 prepared by Julian Templeton and Caroline Barrière (2020).  Version 2 adapted by Caroline Barrière. (2021)

**INTRODUCTION**:  \
We will look at the implementation of a search space.  The code below is largely inspired from the resources provided with the book Artificial Intelligence, Foundations of Computational Agents (https://artint.info/2e/online.html).  The resources corresponding to Chapter 3, on Searching for Solutions, are found here https://artint.info/AIPython/.  

There is even a companion Python book, with explanation of all the python code accompanying the book, as well as a first chapter called **Python for AI** where they give some basics of Python.  See https://artint.info/AIPython/aipython.pdf

The code below is more specifically based on the code in the file *searchProblem.py*.  You do not need to go back to that code. Everything you need is specified in the notebook.  I have modified the original code, and I'm presenting it here in small chunks so as to allow students less familiar with python to go through slowly. 

***HOMEWORK***:  \
Go through the notebook by running each cell, one at a time. If you see an error, you can fix the issue and re-run the cells.  
Look for **(TO DO)** for the tasks that you need to perform.  Once you're done, submit your notebook. Don't forget to put your name at the bottom of the Notebook. Also, RENAME the file to personalize it StudentNumber-LastName-Notebook-2.ipynb.

*The notebook will be marked on 20.  
Each **(TO DO)** has a number of points associated with it.*
***

**1. Definition of a directed graph.**  
A directed graph is made of nodes and arcs. The class below defines an arc that connects two nodes. The first two parameters give the two connected nodes. The other two parameters provide optional cost of an arc (with default to 1) and an optional action of the arc (default to None). The init is the constructor. The assert states a condition that must be true and provides what to do otherwise.

In [1]:
class Arc(object):
    """An arc has a from_node and a to_node node and a (non-negative) cost"""
    
    def __init__(self, from_node, to_node, cost=1, action=None):
        assert cost >= 0, ("Cost cannot be negative for"+
                           str(from_node)+"->"+str(to_node)+", cost: "+str(cost))
        self.from_node = from_node
        self.to_node = to_node
        self.action = action
        self.cost=cost

    def __repr__(self):
        """string representation of an arc"""
        if self.action:
            return str(self.from_node)+" --"+str(self.action)+"--> "+str(self.to_node)
        else:
            return str(self.from_node)+" --> "+str(self.to_node)

Here is an example of a small DAG (Directed Acyclic Graph). 
![Image1](./Image1.png)

We can build a DAG with the Arc class defined above.

In [2]:
# Defining a small dag
dag1 = [Arc('a','b'), Arc('b','c1'), Arc('c1','d'), Arc('b','c2'), Arc('c2','d')]

In [3]:
# print the dag
print(dag1)

[a --> b, b --> c1, c1 --> d, b --> c2, c2 --> d]


The Node class is not defined.  So far, it is assumed that a Node will simply be a string.

**2. Search Space**  
Instead of describing a dag as simply a set of arcs, we can more formally define a class that will represent a search problem which will contain not only the list of arcs, but the explicit list of nodes, the actual start node and the goals.  This class also contains useful methods to "query" the search problem for a node's neighbors, or for determining if a node is a goal or not. It also allows to include a heuristic function, which we will use later.  

In [4]:
class Search_problem_from_explicit_graph():
    """A search problem consists of:
    * a list or set of nodes
    * a list or set of arcs
    * a start node
    * a list or set of goal nodes
    * a dictionary that maps each node into its heuristic value.
    """

    def __init__(self, nodes, arcs, start=None, goals=set(), hmap={}):
        self.neighs = {}
        self.nodes = nodes
        for node in nodes:
            self.neighs[node]=[]
        self.arcs = arcs
        for arc in arcs:
            self.neighs[arc.from_node].append(arc)
        self.start = start
        self.goals = goals
        self.hmap = hmap

    def start_node(self):
        """returns start node"""
        return self.start
    
    def is_goal(self,node):
        """is True if node is a goal"""
        return node in self.goals

    def neighbors(self,node):
        """returns the neighbors of node"""
        return self.neighs[node]

    def heuristic(self,node):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap."""
        if node in self.hmap:
            return self.hmap[node]
        else:
            return 0
        
    def __repr__(self):
        """returns a string representation of the search problem"""
        res=""
        for arc in self.arcs:
            res += str(arc)+".  "
        return res

    def neighbor_nodes(self,node):
        """returns an iterator over the neighbors of node"""
        return (path.to_node for path in self.neighs[node])


Below is an example of a search problem that we might want to represent. Assuming we wish to go from node 'a' to 'h' (highlighted in yellow). Under the figure is the definition.

![Image2](./Image2.png)

In [5]:
# Defining a search space
problemSimple = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j'},
    [Arc('a','b',1), Arc('a','c',1), Arc('b','d',1), Arc('b','e',1),
     Arc('c','f',1), Arc('c', 'g'), Arc('d','h',1), Arc('e','h',1),
     Arc('f', 'e', 1), Arc('f','j',1)],
    start = 'a',
    goals = {'h'})

In [6]:
# Print some information
print(problemSimple.neighbors('a'))

[a --> b, a --> c]


**(TO DO) Q1 - 2 marks**  
Print the start node of the search space *probemSimple*. Test if its node 'g' is a goal node.  Go through its list of nodes to find which one is its goal node.

In [7]:
# ANSWER HERE - Q1
# Print the start node.
print(problemSimple.start_node())
# 
# Test if node 'g' is a goal node.
print(problemSimple.is_goal('g'))
# 
# Go through the list of nodes to find which one is the goal.
for node in problemSimple.nodes:
    goal = repr(problemSimple.goals)
    list_tmp = list(goal)
    if node == list_tmp[2]:
        print(node)
# 

a
False
h


**(TO DO) Q2 - 2 marks** <br>
Complete the definition of the problem space below, according to the following DAG.  The cost are not all equal to 1 now, but are rather given on the arcs.

![Image3](./Image3.png)

In [8]:
# ANSWER HERE - Q2
# problemTest = Search_problem_from_explicit_graph(
#    {'a','b','c','d','e', 'f', 'g', 'h', 'j', 'k', 'l'},
#    .....
#    .....
#    )
problemTest = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j','k','l'},
    [Arc('a','b',6), Arc('a','c',5), Arc('a','d',4), Arc('b','e',2),
     Arc('b','f',4), Arc('c', 'g',3), Arc('d','g',6), Arc('h','j',8),
     Arc('f', 'k', 12), Arc('g','k',7), Arc('j','l',5), Arc('k','l',6)],
    start = 'a',
    goals = {'l'})

**3. Definition of a solution as a path.**  
A solution to a search problem is usually a Path, providing the set of visited nodes and arcs from source node to goal node.  Below a path is defined as either a single node (when arc=None), or a path followed by an arc.  Do not worry too much about all the implementation details in this class.  As your python skills get better, it will become more clear.

In [9]:
# class to represent a path
class Path(object):
    """A path is either a node or a path followed by an arc"""
    
    def __init__(self,initial,arc=None):
        """initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)"""
        self.initial = initial
        self.arc=arc
        if arc is None:
            self.cost=0
        else:
            self.cost = initial.cost+arc.cost

    def end(self):
        """returns the node at the end of the path"""
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node

    def nodes(self):
        """enumerates the nodes for the path.
        This starts at the end and enumerates nodes in the path backwards."""
        current = self
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """enumerates the nodes for the path before the end node.
        This starts at the end and enumerates nodes in the path backwards."""
        if self.arc is not None:
            for nd in self.initial.nodes(): yield nd     # could be "yield from"
        
    def __repr__(self):
        """returns a string representation of a path"""
        if self.arc is None:
            return str(self.initial)
        elif self.arc.action:
            return (str(self.initial)+"\n   --"+str(self.arc.action)
                    +"--> "+str(self.arc.to_node))
        else:
            return str(self.initial)+" --> "+str(self.arc.to_node)

**4.  Definition of a generic searcher**  
We first define a class representing a generic searcher, but with no strategy as to which node will be explored next. 

In [10]:
# (CB) Modified from book
class GenericSearcher():
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()

    
    def initialize_frontier(self):
        self.frontier = [Path(self.problem.start_node())]
        
    def empty_frontier(self):
        return self.frontier == []
       
    ### change the add_to_frontier method --- THIS METHOD NEEDS TO BE IMPLEMENTED FOR DIFFERENT SEARCH STRATEGIES
    def add_to_frontier(self,path):
        raise NotImplementedError

    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            # The next node explored is selected
            # It is implemented with a Pop, which actually removes from top of stack
            path = self.frontier.pop()            
            print(path)
            if self.problem.is_goal(path.end()):    # solution found
                self.solution = path                # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())              
                for arc in neighs:
                    self.add_to_frontier(Path(path,arc))              

**5. Let's implement the three blind searchers.**  
As we saw in class, the behavior of a searcher depends on how the frontier is dealt with.  By modifying the method *add_to_frontier* from the generic searcher, we can implement the 3 types of blind searches saw in the video lectures: depth-first, breadth-first and lowest-cost.  The breadth-first is implemented below.  You are asked to implement the other two.

In [11]:
# (CB) extend the generic searcher to be a Breadth-first searcher.
class BreadthFirstSearcher(GenericSearcher):
    
    def __init__(self, problem):   
        super().__init__(problem)
    def add_to_frontier(self,path):
        # breadth
        self.frontier.insert(0,path)        # HERE, I insert in position 0 (bottom of stack), knowing that I want a FIFO

In [12]:
# Test breadth-first searcher
searcherBreadth = BreadthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherBreadth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> d
a --> b --> e
a --> b --> f
a --> c --> g
a --> d --> g
a --> b --> f --> k
a --> c --> g --> k
a --> d --> g --> k
a --> b --> f --> k --> l
Path found: a --> b --> f --> k --> l with a cost of 28


**(TO DO) Q3 - 2 marks** <br> 
Implement the *add_to_frontier* method for the depth-first search.  And test your implementation.

In [13]:
# ANSWER HERE - Q3 - Part 1 (SINGLE LINE TO DO) (1 mark)
# Depth-first searcher
class DepthFirstSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)
    def add_to_frontier(self,path):
        self.frontier.insert(len(self.frontier),path)
        # depth - put code here
        # ...

In [14]:
# ANSWER HERE - Q3 - Part 2 (Test implementation) (1 mark)
# Test
searcherDepth = DepthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherBreadth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a --> c --> g --> k --> l
Path found: a --> c --> g --> k --> l with a cost of 21


**(TO DO) Q4 - 4 marks** <br> 
Implement the *add_to_frontier* method for the lowest-cost search. And test your implementation.

In [19]:
# ANSWER HERE - Q4 - Part 1 (More complex, must sort so lowest-cost would be the one use in the "pop") (3 marks)
# Lowest Cost searcher
class LowCostSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)
        self.tmp = 10
        
    def getCost(self, path):
        #print(path.arc.from_node)
        ans = path.cost
        return ans
    
    
    def add_to_frontier(self,path):
        # Need to insert path p onto the frontier so the cost order is kept
        # low-cost search  - put code here...
        self.frontier.insert(0,path)
        self.frontier.sort(key=self.getCost, reverse=True)
        #if path.cost < self.tmp:
            #self.frontier.insert(len(self.frontier),path)
            #self.tmp = path.cost
        #else:
            #self.frontier.insert(0,path)

In [20]:
# ANSWER HERE - Q4 - Part 2 (Test implementation) (1 mark)
# Test
searcherLowCost = LowCostSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherLowCost.search()
print(foundPath)
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> d
a --> c
a --> b
a --> c --> g
a --> b --> e
a --> d --> g
a --> b --> f
a --> c --> g --> k
a --> d --> g --> k
a --> c --> g --> k --> l
a --> c --> g --> k --> l
Path found: a --> c --> g --> k --> l with a cost of 21


Let's assume the heuristic function provides an optimistic distance of each node to the goal node 'l'. These distances are encoded in a dictionary call hmap, in the problem definition below, which is an extension to problemTest defined earlier.

In [17]:
problemTestWithHeuristics = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j', 'k', 'l'},
    [Arc('a','b', 6), Arc('a','c', 5), Arc('a','d', 4), 
     Arc('b','e', 2), Arc('b','f', 4),
     Arc('e','h', 5), Arc('h','j', 8), Arc('j','l', 5),
     Arc('f','k', 12), Arc('k','l', 6),
     Arc('c','g', 3), Arc('g','k', 7),
     Arc('d','g', 6)],
    start = 'a',
    goals = {'l'},
     hmap = {'a':20, 'b':12, 'c':15, 'd':14, 'e':9, 'f':13, 'g':10, 'h':11, 'j':5, 'k':6})

**(TO DO) Q5 - 4 marks** <br> 
You must implement and test the A* search algorithm.

In [18]:
# ANSWER HERE - Q5 - Part 1 (3 marks)
# A* searcher
class AStarSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)
        self.hmap1 = problem.hmap
        self.goal = problem.goals
    
    def getCostKey(self, path):
        #if path.arc.to_node == self.goal:
        if path.arc.to_node == 'l':
            ans = path.cost
        else:
            ans = path.cost + self.hmap1[path.arc.to_node]
        return ans
    
    def add_to_frontier(self,path):
        """add path to the frontier with the appropriate cost"""
        # add path to the frontier 
        self.frontier.insert(0,path)
        # sort frontier so that all the elements are ordered from most costly to least costly
        self.frontier.sort(key=self.getCostKey, reverse=True)

In [19]:
# ANSWER HERE - Q5 - Part 2 (1 mark)
# Test
searcherAstar = AStarSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherAstar.search()
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> b --> e
a --> d
a --> c
a --> c --> g
a --> d --> g
a --> c --> g --> k
a --> c --> g --> k --> l
Path found: a --> c --> g --> k --> l with a cost of 21


**(TO DO) Q6 - 4 marks** <br> 
You must implement and test the Best-First Search algorithm.

In [20]:
# ANSWER HERE - Q6 - Part 1 (3 marks)
# Best first searcher
class BestFirstSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):
        super().__init__(problem)
        self.hmap1 = problem.hmap
            
    def getCostKey(self, path):
        if path.arc.to_node == 'l':
            ans = 0
        else:
            ans = self.hmap1[path.arc.to_node]
        return ans
    def add_to_frontier(self,path):
        self.frontier.insert(0,path)
        self.frontier.sort(key=self.getCostKey, reverse=True)

In [21]:
# ANSWER HERE - Q6 - Part 2 (1 mark)
# Test
searcherBestFirst = BestFirstSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherBestFirst.search()
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> b --> e
a --> b --> e --> h
a --> b --> e --> h --> j
a --> b --> e --> h --> j --> l
Path found: a --> b --> e --> h --> j --> l with a cost of 26


**(TO DO) Q7 - 2 marks**\
Now that you have 5 algorithms implemented, 3 blind ones and 2 global heuristic ones, discuss: \
(a) among breadth-first and depth-first, which one performed better (in terms of finding a lower cost path and in terms of number of nodes explored) and was that as low as the result obtained with the lowest-cost first search, and \
(b) among the 2 heuristic searches, which one performed better (in terms of finding a lower cost path).

ANSWER HERE - Q7   
(a)Breadth-first search can have lowest average of time in terms of finding lower cost path. But in some special case, depth-first can have smallest cost of finding lower cost path.

(b)A* searches would be better, because it combine both less cost and best first search

***SIGNATURE:***
My name is Tan Chen.
My student number is 300072995.
I certify being the author of this assignment.